In [2]:
import os
import subprocess
import numpy as np
import xarray as xr
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# configuration
resolution = "p05"
region = [-180,180,-60,90]
data_path = "/share/home/dq076/bedrock/data/"
shp_path = '/share/home/dq076/bedrock/data/Shp/'

size = 0.05

In [3]:
ds = xr.open_dataset(f'{data_path}diff/diff.nc') # ensemble mean ET - ensemble mean P, 8-day, 0p1, 2003.01.01-2020.12.31
current_diff = ds['diff']
ds2 = xr.open_dataset(f'{data_path}SC/SC.nc') # snowcover>10 -> 0, snowcover<=10 -> 1, 8-day, 0p1,  2003.01.01-2020.12.31
snowf = ds2['SC']
ds3 = xr.open_dataset(f'{data_path}Ssoil/Ssoil.nc') # soil water storage capacity, 0p1
ssoil = ds3['Ssoil']
print(data_path)

/share/home/dq076/bedrock/data/


In [4]:
# Obtain shape and duration
shape = current_diff.isel(time=0).shape
time_len = len(ds.time)

print(time_len)

# Initialize variables
current_cwd = np.zeros(shape) 
sr = np.zeros(shape)
sbedrock = np.zeros(shape)

print(ds['time'])

828
<xarray.DataArray 'time' (time: 828)> Size: 7kB
array(['2003-01-01T00:00:00.000000000', '2003-01-09T00:00:00.000000000',
       '2003-01-17T00:00:00.000000000', ..., '2020-12-10T00:00:00.000000000',
       '2020-12-18T00:00:00.000000000', '2020-12-26T00:00:00.000000000'],
      shape=(828,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 7kB 2003-01-01 2003-01-09 ... 2020-12-26
Attributes:
    standard_name:  time
    axis:           T


In [5]:
for i in range(time_len):
    print(f"Processing time index: {i}")
    day_stt = 8*i+1-3*(i//46)+((i//46)+2)//4
    day_end = 8*(i+1)+1-3*((i+1)//46)+(((i+1)//46)+2)//4-1
    day_duration = day_end-day_stt+1
    j = i//46
    year = j+2003
    print(year)
    print(f"the period {i+1:3} day from {day_stt:4} to {day_end:4}")
    print(f"the period {i+1:3} day is {day_duration:1}")

    # Calculate current delta_tn, cwd and sr
    current_delta_tn = current_diff.isel(time=i).values * snowf.isel(time=i).values
    last_cwd = current_cwd
    current_cwd = np.where(current_delta_tn >= 0, current_cwd + current_delta_tn, 0)
    sr = np.maximum(sr, current_cwd)

sbedrock = np.where(sr > ssoil, sr - ssoil, 0)

Processing time index: 0
2003
the period   1 day from    1 to    8
the period   1 day is 8
Processing time index: 1
2003
the period   2 day from    9 to   16
the period   2 day is 8
Processing time index: 2
2003
the period   3 day from   17 to   24
the period   3 day is 8
Processing time index: 3
2003
the period   4 day from   25 to   32
the period   4 day is 8
Processing time index: 4
2003
the period   5 day from   33 to   40
the period   5 day is 8
Processing time index: 5
2003
the period   6 day from   41 to   48
the period   6 day is 8
Processing time index: 6
2003
the period   7 day from   49 to   56
the period   7 day is 8
Processing time index: 7
2003
the period   8 day from   57 to   64
the period   8 day is 8
Processing time index: 8
2003
the period   9 day from   65 to   72
the period   9 day is 8
Processing time index: 9
2003
the period  10 day from   73 to   80
the period  10 day is 8
Processing time index: 10
2003
the period  11 day from   81 to   88
the period  11 day is 

In [6]:
output_ds = xr.Dataset({'Sr': (('lat', 'lon'), sr)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds.to_netcdf(f'{data_path}Sr_tmp1.nc')

output_ds1 = xr.Dataset({'Sbedrock': (('lat', 'lon'), sbedrock)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds1.to_netcdf(f'{data_path}Sbedrock_tmp1.nc')